In [ ]:
# this script takes the clean dataset gets the data for the current month
# Created by Skye
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from fbprophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
from datetime import datetime
from dateutil.parser import parse
from dateutil import relativedelta
from tqdm import tqdm
import json
    
# number of months between two dates
def find_pred_size (df:pd.DataFrame) -> int:
    assert 'ds' in df.columns, "'ds' column needs to be in the input DataFrame"
    today = datetime.now()
    r = relativedelta.relativedelta(today, df['ds'].max())
    years = r.years
    months = r.months
    return int((years * 12) + months)

#do we want to rent or buy
def interest_dataset(data, rent_or_buy, zip_code_of_interest):
    #based on the input which column do we want to look at
    # we will do both as long as data is present for each zip code
    if rent_or_buy == 'rent':   
        renamed_dataset = data.rename(columns={ 'Zri_MultiFamilyResidenceRental': 'y'})
    else:
        renamed_dataset = data.rename(columns={'ZHVI_SingleFamilyResidence': 'y'})
        
    return renamed_dataset

#actual prediction model
def find_value_today(data, prediction_size):
    try:
        my_model = Prophet(interval_width=0.95, 
                           yearly_seasonality= 3, 
                           weekly_seasonality=False, 
                           daily_seasonality=False)
        #my_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
        my_model.fit(data)
        future = my_model.make_future_dataframe(periods=prediction_size, freq='M')
        forecast = my_model.predict(future)
        my_forcast_data = forecast[['ds', 'yhat']]
        value_today = forecast['yhat'].iloc[-1]
        appr_data = forecast["trend"]
    except:
        value_today = 0
    return value_today, appr_data

def cal_appre_rate(df, appr_data):
    old_value = appr_data.iloc[-37]
    print(old_value)
    new_value = appr_data.iloc[-1]
    print(new_value)
    rate = np.exp(np.log(new_value/old_value)/3) - 1.0
    return round(rate*100, 2)

def make_main(my_data, zip_code_of_interest):
    #for each zipcode
    clean_data = my_data.loc[my_data['ZipCode'] == zip_code_of_interest]
    
    #change date column to the correct format
    prediction_size_rent = find_pred_size(clean_data)
    prediction_size_buy  = find_pred_size(clean_data)
    
    #zipcode we want data for
    # #need to check if region name is always the same...
    rent_dataset = interest_dataset(clean_data, 'rent', zip_code_of_interest)
    buy_dataset  = interest_dataset(clean_data, 'buy', zip_code_of_interest)
   
    #predict
    rent_value, appr_data_rent = find_value_today(rent_dataset, prediction_size_rent)
    #print(rent_value)
    buy_value, appr_data_buy  = find_value_today(buy_dataset, prediction_size_buy)
    #print(buy_value)
    
    #appreciation rate
    appr_rate_buy = cal_appre_rate(buy_dataset, appr_data_buy)
    
    #round values
    rent_value = int(rent_value)
    buy_value = round(buy_value/1000, 1)
    
    return {str(zip_code_of_interest): {"buy": f'${buy_value:,}k', "rent": f'${rent_value:,}/month', "appr_rate" : f'{appr_rate_buy}%'}}

    

In [ ]:
data = pd.read_pickle('../data/interim/interpolated_fillnaTime_df.pickle')
data.tail()

In [ ]:

#clean_data['Date_timestamp']= clean_data['Date'].apply(lambda _: datetime.strptime(_,"%Y-%m-%d"))

In [ ]:
clean_data = data


In [ ]:
#df = rent_dataset
#df.loc[df['ds'] == df['ds'].max(), 'y'].iloc[0]
#rent_dataset.tail()

In [ ]:
test_zip = 96090

test_df = data[data['ZipCode'] == test_zip]
prediction_size_rent = find_pred_size(test_df)
prediction_size_buy  = find_pred_size(test_df)
    
#zipcode we want data for
# #need to check if region name is always the same...
rent_dataset = interest_dataset(test_df, 'rent', test_zip)
buy_dataset  = interest_dataset(test_df, 'buy', test_zip)

my_model = Prophet(interval_width=0.95, 
                           yearly_seasonality= 3, 
                           weekly_seasonality=False, 
                           daily_seasonality=False)

In [ ]:
# BUY

#my_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
my_model.fit(buy_dataset)

#future_b = my_model.make_future_dataframe(periods=prediction_size_buy, freq='M')
forecast_b = my_model.predict(future_b)

my_forcast_data_b = forecast_b[['ds', 'yhat']]
#value_today = forecast['yhat'].iloc[-1]
        #date_values = forecast['ds'].tolist()
        #value_today = forecast['yhat'].tolist()
        #print(value_today)
print(my_forcast_data_b)

In [ ]:
my_model.plot(forecast_b)

In [ ]:
my_model.plot_components(forecast_b)

In [ ]:
new_val = forecast_b["trend"].iloc[-1]
old_val = forecast_b["trend"].iloc[-37]

In [ ]:
# RENT
#my_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
my_model.fit(rent_dataset)

future_r = my_model.make_future_dataframe(periods=prediction_size_rent, freq='M')
forecast_r = my_model.predict(future_r)

#my_model.fit(rent_dataset)
#future = my_model.make_future_dataframe(periods=prediction_size_rent, freq='M')
#forecast = my_model.predict(future)
my_forcast_data_r = forecast_r[['ds', 'yhat']]
#value_today = forecast['yhat'].iloc[-1]
        #date_values = forecast['ds'].tolist()
        #value_today = forecast['yhat'].tolist()
        #print(value_today)
print(my_forcast_data_r)

In [ ]:
my_model.plot(forecast_r)

In [ ]:
#fig1 = my_model.plot(forecast)

In [ ]:
#fig2 = my_model.plot_components(forecast)

In [ ]:
#a = forecast["trend"]
#print(a)

In [ ]:
test_list = [95246, 95113, 96090, 96134, 91381, 92086, 92651, 92661, 95321, 95327, 95370,95383, 95457, 95461, 96101, 93544, 93553, 92268, 92285, 92321, 92347, 92358, 92539, 92617, 93517, 93646, 95306, 95322, 95333, 95335, 95372, 95379, 95389, 95912, 95982, 96103, 96107, 96093, 95589, 96041]
result_dict={}
for i in tqdm(test_list):
    #print(i)
    result = make_main(clean_data, i)
    #print(result)
    result_dict.update(result)
    

In [ ]:
from pprint import pprint
pprint(result_dict)

In [ ]:
cal_zip_list = clean_data['ZipCode'].unique().tolist()
len(cal_zip_list)

In [ ]:
result_dict={}
for i in tqdm(cal_zip_list):
    result = make_main(clean_data, i)
    result_dict.update(result)

In [ ]:
print(result_dict)

In [ ]:
my_dict = json.dumps(result_dict)

In [ ]:
print(my_dict)

In [ ]:
test_dict = json.loads(my_dict)

In [ ]:
print(test_dict)

In [ ]:
df = pd.DataFrame(test_dict).T
df.head()
df.to_csv('../data/interim/rent_buy_zip_appr.csv')

In [ ]:
df.head()

In [ ]:
import datacompy


In [ ]:
df1 = pd.read_csv('../data/interim/rent_buy_zip_interp.csv')
df2 = pd.read_csv('../data/interim/rent_buy_zip.csv')

In [ ]:
df2.rename(columns={'Unnamed: 0':'Zipcode'}, inplace=True)
df2.head()
df1.rename(columns={'Unnamed: 0':'Zipcode'}, inplace=True)
df1.head()

In [ ]:
compare = datacompy.Compare(
    df1,
    df2,
    join_columns= 'Zipcode',  #You can also specify a list of columns eg ['policyID','statecode']
    df1_name='Interp', #Optional, defaults to 'df1'
    df2_name='OG' #Optional, defaults to 'df2'
)

In [ ]:
print(compare.report())

In [ ]:
df['buy'] = df['buy'].map(lambda x: x.lstrip('$').rstrip('k'))
df['rent'] = df['rent'].map(lambda x: x.lstrip('$').rstrip('/month'))
df['appr_rate'] = df['appr_rate'].map(lambda x: x.rstrip('%'))
#df['buy'] = df['buy'].astype(int)
df['buy']=df['buy'].str.replace(',','')
df['rent']=df['rent'].str.replace(',','')
df.head()

In [ ]:
df['buy'] = pd.to_numeric(df['buy'])
#df['rent'] = df['rent'].astype(int)
#df['buy'] = df['buy'].astype(int)
df.head()

In [ ]:
#type(df['buy'].values[0])
#sum(n < 0.0 for n in df.values.flatten())

In [ ]:
(n < 0.0 for n in df.values.flatten())

In [ ]:
myData = df.fillna(0).query('buy < 0 or rent < 0')
myData

In [ ]:
bad_zips = myData.index.tolist()
print(bad_zips)

In [ ]:
test = data.fillna(0).query('ZHVI_SingleFamilyResidence < 0 or Zri_MultiFamilyResidenceRental < 0')
test.head()

In [ ]:
data.head()

In [ ]:
syl_dict={}
for i in tqdm(cal_zip_list): 
    result = find_pred_size(data)
    syl_dict[i] = result

In [ ]:
print(syl_dict)

In [ ]:
df2.head()

In [ ]:
not_int = pd.read_pickle('../data/interim/ca-zip-w-city-ts.pickle')
#not_int.rename(columns={'Date': 'ds'}, inplace=True)
not_int.head()
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month
def pred_this (ts_data):
    date_today = datetime.today().strftime('%Y-%m-%d')
    latest_date = max(dt for dt in ts_data['Date'] if dt < date_today)
    prediction_size = diff_month(datetime.strptime(date_today, "%Y-%m-%d"),datetime.strptime(latest_date, "%Y-%m-%d"))
    return(prediction_size)

In [ ]:
syl_dict={}
for i in tqdm(cal_zip_list): 
    result = pred_this(not_int)
    syl_dict[i] = result

In [ ]:
print(syl_dict)

In [ ]:
len(syl_dict)